# Introduction
Previously we observed that both the ACEDIA and SELCYS reactions have an ambiguosly defined electron acceptor associated to them. Here I will look further into each reaction, and per Ben's advice solve the issues with mass balance here.

In [2]:
import cameo
import pandas as pd
import cobra.io
from cobra import Reaction, Metabolite

In [13]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

## ACEDIA
This reaction converts acetolactate to diacetyl and co2 in a spontaneous reaction. It is associated with an electron acceptor but there is no information about which acceptor this may be. Therefore, Ben recommended that we keep the acceptor as an ambiguous entity, and allow it to accept electrons in this reaction.

Then we can add a paired reaction where the acceptor donates its electrons and proton to NAD, regenerating NADH. For this specific reaction we chose this co-factor as it is the most logical considering the role this reaction plays. If ever more information about this reaction appears one can change it.

Also it should be noted that whenever a pathway is analyzed that relies on the end of this pathway, to investigate what the effect would be of removing the cofactor requirement at all. To analyze the role of this in cofactor (re)generation.


In [14]:
model.add_metabolites(Metabolite(id='acc_c'))

In [15]:
model.metabolites.acc_c.name = 'Acceptor'
model.metabolites.acc_c.formula = 'R'
model.metabolites.acc_c.compartment = 'c'
model.metabolites.acc_c.charge = 0
model.metabolites.acc_c.annotation['kegg.compound'] = 'C00028'
model.metabolites.acc_c.annotation['chebi'] = 'CHEBI:15339'

In [16]:
model.metabolites.hacc_c.formula = 'HR'
model.metabolites.hacc_c.charge = -1

In [17]:
model.metabolites.hacc_c.annotation['kegg.compound'] = 'C00030'
model.metabolites.hacc_c.annotation['chebi'] = 'CHEBI:17499'

In [18]:
model.reactions.ACEDIA.add_metabolites({
    model.metabolites.acc_c:-1,
    model.metabolites.h_c: -1
})

In [19]:
#make irreversible as it is spontaneous decarboxylation
model.reactions.ACEDIA.bounds = (0,1000)

In [20]:
#Add regeneration reactions
model.add_reaction(Reaction(id='ACCR'))

In [21]:
model.reactions.ACCR.name = 'acceptor regeneration'

In [22]:
model.reactions.ACCR.notes['NOTES'] = 'Assumed regeneration with NADH'
model.reactions.ACCR.annotation['sbo'] = 'SBO:0000176'

In [23]:
model.reactions.ACCR.add_metabolites({
    model.metabolites.hacc_c: -1,
    model.metabolites.acc_c:1,
    model.metabolites.nad_c:-1,
    model.metabolites.nadh_c:1
})

In [ ]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

In [66]:
model.reactions.ACCR

Reaction identifier,ACCR
Name,acceptor regeneration
Memory address,0x011819639a88
Stoichiometry,hacc_c + nad_c --> acc_c + nadh_c Hydrogen-acceptor + NAD+ --> Acceptor + NADH
GPR,
Lower bound,0.0
Upper bound,1000.0


In [68]:
model.reactions.ACCR.check_mass_balance()

{}